In [1]:

import os

In [3]:
%pwd

'/home/mohammadrasel/project/another/Kidney-Disease-Classification-Deep-Learning-Project/research'

In [44]:
os.chdir("../")

In [45]:
%pwd

'/home/mohammadrasel/project/another/Kidney-Disease-Classification-Deep-Learning-Project'

In [72]:


from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_base_model_path:Path
    params_image_size:list
    params_learning_rate:float
    params_include_top:bool
    params_weights:str
    params_classes:int
    

    
    
    

In [73]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [74]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
        
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [75]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf


In [76]:
class PrepareBaseModel:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config=config
        
    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
                input_shape=self.config.params_image_size,
                weights=self.config.params_weights,
                include_top=self.config.params_include_top
                
        )
        self.save_model(path=self.config.base_model_path, model=self.model)
        
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable=False
        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable=False
        
        flatten_in=model.output
        x=tf.keras.layers.Flatten()(flatten_in)
        prediction=tf.keras.layers.Dense(units=classes, activation="softmax")(x)
        
        full_model=tf.keras.Model(inputs=model.input, outputs=prediction)
        # I can modify Adam instead of SGD 
        
        full_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate), loss="categorical_crossentropy", metrics=["accuracy"])
        
        return full_model
    
    def update_base_model(self):
        self.full_model=self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

In [80]:
try:
    config=ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    print(prepare_base_model_config)
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    
except Exception as e:
    raise e 

[2026-02-09 23:20:11,851: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-02-09 23:20:11,855: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-09 23:20:11,857: INFO: common: created directory at: artifacts]
[2026-02-09 23:20:11,858: INFO: common: created directory at: artifacts/prepare_base_model]
PrepareBaseModelConfig(root_dir=PosixPath('artifacts/prepare_base_model'), base_model_path=PosixPath('artifacts/prepare_base_model/base_model.h5'), updated_base_model_path=PosixPath('artifacts/prepare_base_model/base_model_updated.h5'), params_image_size=BoxList([224, 224, 3]), params_learning_rate=0.01, params_include_top=False, params_weights='imagenet', params_classes=2)
[2026-02-09 23:20:12,190: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
